# Лабораторная работа №4
БЭКПРОПАГЕЙШН

Выполнили студенты Зимин Андрей Валерьевич, Жилин Андрей Игроевич

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numpy.f2py.auxfuncs import throw_error

from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as rec
from sklearn.metrics import precision_score as pre
from sklearn.model_selection import train_test_split
from matplotlib.ticker import MaxNLocator
import qrcode
import numpy as np
from matplotlib import pyplot as plt
import random
import string
from typing import Callable, List, Union
from numpy.typing import NDArray
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as rec
from sklearn.metrics import precision_score as pre
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from matplotlib.ticker import MaxNLocator

## Задание в классе. Предварительное исследование данных.

1. Загрузить данные из файлов «vinequality-red.csv» и «vinequality-white.csv» об образцах красного и белого вина, соответственно.
2. На основе загруженных данных формировать 3 датасета:
Параметры и качество белого вина (12 признаков, 4898 образцов) по данным второго файла,
Параметры и качество вина Vinho Verde (12 признаков, 6497 образцов), по данным обоих файлов.
Параметры и качество вина Vinho Verde (13 признаков, 6497 образцов), по данным обоих файлов, введя бинарный признак для отличия красного и белого вина.
Распечатать размерность и первые 5 строк каждого датасета.
3. Построить гистограмму распределения значения признака «качество вина» по каждому датасету.
4. Для каждого датасета вычислить матрицу парных корреляций всех признаков и построить её тепловую карту (по примеру на картинке).

5. Заменить значения входных признаков каждого датасета их стандартизованными значениями, т.е. выполнить Z-стандартизацию по формуле




### Задание 1.1

Загрузить данные из файлов «vinequality-red.csv» и «vinequality-white.csv» об образцах красного и белого вина, соответственно.


In [ ]:
df_redwine = pd.read_csv("./data/winequality-red.csv", sep=";")
df_redwine.head()

# Задание 1.2
На основе загруженных данных формировать 3 датасета:
A. Параметры и качество белого вина (12 признаков, 4898 образцов) по данным второго файла,

In [ ]:
df_whitewine = pd.read_csv("./data/winequality-white.csv", sep=";") # A-датасет
print(df_whitewine.info())
df_whitewine.head()

B. Параметры и качество вина Vinho Verde (12 признаков, 6497 образцов), по данным обоих файлов.

In [ ]:
df_redwhite = pd.concat([df_redwine, df_whitewine], ignore_index=True)
print(df_redwhite.info())
df_redwhite.head()

In [ ]:
temp_df_whitewine = df_whitewine.copy()
temp_df_redwine = df_redwine.copy()
temp_df_whitewine['is_red'] = pd.Series([False]*len(df_redwine), dtype="bool")
temp_df_redwine['is_white'] = pd.Series([False]*len(df_redwine), dtype="bool")
temp_df_whitewine['is_white'] = pd.Series([True]*len(df_redwine), dtype="bool")
temp_df_redwine['is_red'] = pd.Series([True]*len(df_redwine), dtype="bool")

temp_df_redwine.head()

Загрузили датасет. Действительно 7 признаков и 333 строки

In [ ]:
df = pd.concat([df_redwine, df_whitewine], ignore_index=True)


species            - категориальный признак  
island             - категориальный признак  
bill_length_mm     - числовой признак  
bill_depth_mm      - числовой признак  
flipper_length_mm  - числовой признак  
body_mass_g        - числовой признак  
sex                - бинарный признак   

Перед обучением нужно будет закодировать категориальные признаки one-hot encoding

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
df.describe()

Видим, что сильно выделяется признак массы тела, он на 2-3 порядка больше остальных признаков. Перед обучением было бы неплохо провести нормализацию

In [ ]:
category_columns = ['species', 'island', 'sex']

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
#for i, category in enumerate(category_columns):
#   ax[i].bar()

In [ ]:
class Perceptron:
  def __init__(self, hidden_layers : List[int],
               activations: List[Callable],
               eta:Union[int, float] = 1,
               n_epochs: int = 100,
               random_weights=None,
               random_state=None):
    np.random.seed(random_state)
    random.seed(random_state)
    self.is_fitted = False
    self.layers = []
    self.epochs = n_epochs
    self.eta = eta
    self.errors = []
    l = hidden_layers + [1]
    for i in range(len(hidden_layers)):
      self.layers.append(Layer((l[i]+1, l[i+1]), activations[i], i + 1, random_weights=random_weights))

  @staticmethod
  def relu(x):
      return np.maximum(0, x)
  @staticmethod
  def step(x):
      return (x >= 0).astype(np.float64)

  @staticmethod
  def sigmoid(x):
    return np.where(x >= 0,
                   1 / (1 + np.exp(-x)),
                   np.exp(x) / (1 + np.exp(x)))

  @staticmethod
  def shuffle(X, y):
    n = len(y)
    a = [(random.random(), X[i, :], y[i]) for i in range(n)]
    a.sort()
    new_X = np.array([a[i][1] for i in range(n)])
    new_y = np.array([a[i][2] for i in range(n)])
    return new_X, new_y

  @staticmethod
  def get_grad(activation):
      activation = activation.pyfunc
      if activation == Perceptron.sigmoid:
          s = Perceptron.sigmoid
          return lambda x: s(x) * (1 - s(x))
      return lambda x: (x >= 0).astype(np.int64)


  def predict(self, train_sample: NDArray, logging: bool = False) -> NDArray:
    result = np.zeros(train_sample.shape[0])
    for i in range(train_sample.shape[0]):
      x = train_sample[i, :]
      for layer in self.layers:
        x = np.append(x, values=[1]) # добавление свободного коэффициента
        x = layer.forward(x, logging)

      result[i] = x[0]
      if logging:
        print(result[i])
    return result

  def train(self, train_sample: NDArray,
          train_ans: NDArray,
          batch_size: int = 32,  # Размер пакета (можно менять)
          logging=False,
          activation=np.sign,
          random_weights=None) -> list[float]:
    if not self.is_fitted:
        self.layers = [Layer((train_sample.shape[1]+1,
                            self.layers[0].size[0]-1),
                            activation,
                            0,
                            random_weights=random_weights)] + self.layers
        self.is_fitted = True

    n_samples = train_sample.shape[0]
    for epoch in range(self.epochs):
        train_sample, train_ans = Perceptron.shuffle(train_sample, train_ans)
        error = 0

        # Разбиваем данные на пакеты
        for batch_start in range(0, n_samples, batch_size):
            batch_end = min(batch_start + batch_size, n_samples)
            batch_X = train_sample[batch_start:batch_end, :]
            batch_y = train_ans[batch_start:batch_end]

            # Инициализируем накопители градиентов для каждого слоя
            weight_updates = [np.zeros_like(layer.w) for layer in self.layers]
            batch_error = 0

            # Обрабатываем каждый пример в пакете
            for i in range(batch_X.shape[0]):
                x = batch_X[i, :]
                target = batch_y[i]

                # Прямое распространение (forward pass)
                activations = [x]
                for layer in self.layers:
                    x = np.append(x, values=[1])
                    x = layer.forward(x, logging)
                    activations.append(x)

                predicted = x[0]
                delta = target - predicted
                batch_error += abs(delta)

                # Обратное распространение (backward pass)
                delta_next = delta
                for layer_idx in range(len(self.layers)-1, -1, -1):
                    layer = self.layers[layer_idx]
                    layer_input = activations[layer_idx]
                    delta_next, weight_update = layer.backprop(delta_next, layer_input, self.eta)
                    weight_updates[layer_idx] += weight_update  # Накопление градиентов

            # Усредняем градиенты и обновляем веса
            for layer, update in zip(self.layers, weight_updates):
                layer.w += (self.eta / batch_size) * update  # Усреднённое обновление

            error += batch_error

        self.errors.append(error)
    return self.errors


class Layer:
  def __init__(self,
               size: tuple[int, int],
               activation: Callable,
               index: int,
               value: Union[int, float]=0,
               random_weights=None):
    self.size = size
    self.w = np.full(size, value, dtype=np.float64)
    if random_weights is not None:
      self.w = np.random.uniform(random_weights[0], random_weights[1], self.size)
    self.activation = np.vectorize(activation)
    self.i = index
    self.last_result = np.array([])
    self.last_x = np.array([])
    self.last_m = np.array([])

  def backward(self, value) -> None:
    if value > 0:
      # надо увеличить те веса, где нет активации,
      # но должна быть активация
      d = (self.last_x>0)*value
      d = np.repeat(np.array([d]).T, self.w.shape[1], axis=1)
      self.w = self.w + d
    if value < 0:
      # надо уменьшить те веса,
      # где активации быть не должно
      d = (self.last_x>0)*value
      d = np.repeat(np.array([d]).T, self.w.shape[1], axis=1)
      self.w = self.w + d
  def backprop(self, delta_prev, layer_input, eta):
    # Получаем взвешенную сумму (последний расчёт в forward)
    m = self.last_m

    # Вычисляем производную активационной функции
    grad = Perceptron.get_grad(self.activation)(m)

    # delta для текущего слоя = delta_prev * производная активации
    delta = delta_prev * grad

    # Добавляем bias-единицу к входному вектору (если её ещё нет)
    layer_input_with_bias = np.append(layer_input, 1)

    # Возвращаем ошибку для предыдущего слоя (без учёта bias-весов)
    delta_next = delta @ self.w[:-1].T

    # Возвращаем градиент для накопления (без обновления весов)
    weight_update = np.outer(layer_input_with_bias, delta)

    return delta_next, weight_update

  def forward(self, x, logging) -> NDArray:
    if logging:
      print(f"Слой №{self.i+1}")
      print(f"Сенсоры  : {x}")
      print(f"Размер: {self.size}")

    m = np.dot(x, self.w)
    result = self.activation(m)
    self.last_result = result
    self.last_x = x
    self.last_m = m
    if logging:
      print(f"Сумматор : {m}")
      print(f"Активация: {result}")
      print(f"Результат размера {result.shape}")

    return result

def count_metrics(predicted : NDArray[bool], real: NDArray[bool], label='') -> None:
    print(f'{label}: ')
    print('Accuracy: ', acc(predicted, real))
    print('Recall: ', rec(predicted, real))
    print('Precision: ', pre(predicted, real))


def get_features(data, target, target_name, features) -> (NDArray, NDArray):

    data = data[features + [target]]
    data.loc[:, target] = (data[target] == target_name)
    data_y = data[[target]].to_numpy().reshape(1, -1)[0] * 1
    data_x = data[features].to_numpy()

    return data_x, data_y


def build_errors(errors: list[float], perceptron: Perceptron) -> None:
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.plot(list(range(len(errors))), errors, marker='o', linestyle='dashed')

    plt.xlabel("Эпохи")
    plt.ylabel("Количество ошибок за эпоху")
    plt.title("Обучение нейросети\nбинарной классификации на числовых признаках")
    print("Размерность слоёв: ", *[i.size for i in  perceptron.layers])

In [ ]:
def print_contour(model, X, y, main_1=0, main_2=1, supp=None, columns_num=None):
    # 1. Генерируем сетку для 2-х выбранных признаков
    if supp is None:
        supp = [2, 3]
    if columns_num is None:
        columns_num = [''] * 2
    x_min, x_max = X[:, main_1].min() - 1, X[:, main_1].max() + 1
    y_min, y_max = X[:, main_2].min() - 1, X[:, main_2].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

    # 2. Создаём массив точек в 4-х мерном пространстве, но только для выбранных 2-х признаков
    mean_vals = np.mean(X, axis=0)  # Средние значения для всех 4-х признаков
    fixed_features = mean_vals[supp]  # Средние значения для 3-го и 4-го признаков

    grid_points_4d = np.zeros((xx.ravel().shape[0], 2+len(supp)))
    grid_points_4d[:, [main_1, main_2]] = np.column_stack((xx.ravel(), yy.ravel()))  # 2 выбранных признака
    grid_points_4d[:, [supp]] = fixed_features  # Остальные 2 признака - средние значения

    # 3. Предсказываем классы для сеточных точек в 4-х мерном пространстве
    Z = model.predict(grid_points_4d)
    Z = Z.reshape(xx.shape)

    # 4. Визуализация
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
    plt.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], linewidths=[1, 2, 1])

    # Отображаем точки данных
    plt.scatter(X[:, main_1], X[:, main_2], c=y, cmap='coolwarm', edgecolor='k')
    plt.xlabel(columns_num[main_1])
    plt.ylabel(columns_num[main_2])
    plt.title('Проекция линий уровня персептрона на 2D')
    plt.show()

